# MyFlixer Project

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
url = 'https://myflixerz.to/home'

In [5]:
response = requests.get(url)

In [6]:
response

<Response [200]>

In [20]:
response.text

'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\n<head>\n    <title>MyFlixer - Watch movies and Series online free in Full HD on MyFlixer</title>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>\n\n    <meta name="robots" content="index, follow">\n    <meta name="revisit-after" content="1 days">\n\n<meta http-equiv="content-language" content="en"/>\n<link rel="dns-prefetch" href="//www.google-analytics.com">\n<link rel="dns-prefetch" href="//www.gstatic.com">\n<link rel="dns-prefetch" href="//fonts.gstatic.com">\n<link rel="dns-prefetch" href="//cdnjs.cloudflare.com">\n<link rel="dns-prefetch" href="//static.zdassets.com">\n\n<link rel="dns-prefetch" href="//connect.facebook.net">\n<link rel="dns-prefetch" href="//graph.facebook.com">\n<meta name="description" content="Full Movies Online Streaming and Tv Shows Streaming At no cost - Fast and Free with Great support on many devices - Chromecast supported - Safe Private streaming

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [21]:
soup

<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>MyFlixer - Watch movies and Series online free in Full HD on MyFlixer</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="index, follow" name="robots"/>
<meta content="1 days" name="revisit-after"/>
<meta content="en" http-equiv="content-language">
<link href="//www.google-analytics.com" rel="dns-prefetch"/>
<link href="//www.gstatic.com" rel="dns-prefetch"/>
<link href="//fonts.gstatic.com" rel="dns-prefetch"/>
<link href="//cdnjs.cloudflare.com" rel="dns-prefetch"/>
<link href="//static.zdassets.com" rel="dns-prefetch"/>
<link href="//connect.facebook.net" rel="dns-prefetch"/>
<link href="//graph.facebook.com" rel="dns-prefetch"/>
<meta content="Full Movies Online Streaming and Tv Shows Streaming At no cost - Fast and Free with Great support on many devices - Chromecast supported - Safe Private streaming" name="description">
<meta content="myflixe

In [8]:
df = pd.DataFrame(columns=['Title', 'Quality', 'Cast', 'Production', 'Description'])

In [9]:
df

,Title,Quality,Cast,Production,Description


In [10]:
def get_movies(html):
    all_movies = html.find_all('div', class_='flw-item')
    return all_movies

In [11]:
def movie_info(movie, all_info):
    title_tag = movie.find('h3', class_='film-name')
    title = title_tag.text.replace('\n', '')
    quality = (lambda x: x.find('div', class_='pick film-poster-quality').text if x.find('div', class_='pick film-poster-quality') else '')(movie)
    
    link = title_tag.find('a').get('href')
    cast_names, prod_team, movie_desc = movie_page(link)
    if title not in df['Title'].values:
        df.loc[len(df)] = [title, quality, ', '.join(cast_names), ', '.join(prod_team), movie_desc]

In [12]:
def movie_page(url):
    prefix = f'https://myflixerz.to{url}'
    res = requests.get(prefix)
    page = BeautifulSoup(res.text, 'html.parser')
    elements = page.find_all('div', class_='elements')
    actors = list(cast(elements))
    prod = list(production(elements))
    descr = description(page)
    return actors, prod, descr

In [13]:
def cast(movie_elements):
    cast_div = movie_elements[0].find('div', class_='col-xl-5 col-lg-6 col-md-8 col-sm-12')
    cast_links = cast_div.find_all('a', href = lambda name: name and '/cast/' in name)
    cast_names = [x.text for x in cast_links]
    return cast_names

In [14]:
def production(movie_elements):
    production_div = movie_elements[0].find('div', class_='col-xl-6 col-lg-6 col-md-4 col-sm-12')
    production_links = production_div.find_all('a', href = lambda name: name and '/production/' in name)
    production_names = [x.text for x in production_links]
    return production_names

In [15]:
def description(movie_page):
    desc = movie_page.find_all('div', class_='description')[0].text
    desc = desc.replace('\n', '')
    return desc

In [16]:
movies = get_movies(soup)

In [17]:
for movie in movies:
    movie_info(movie, df)

In [18]:
df

,Title,Quality,Cast,Production,Description
0,Moana 2,CAM,"Auli'i Cravalho, Dwayne Johnson, Rachel House,...","Walt Disney Pictures, Walt Disney Animation St...",After receiving an unexpected call from he...
1,Venom: The Last Dance,CAM,"Chiwetel Ejiofor, Peggy Lu, Alanna Ubach, Rhys...","Columbia Pictures, Pascal Pictures, Matt Tolma...",Eddie and Venom are on the run. Hunted by ...
2,Absolution,HD,"Josh Drennen, Jimmy Gonzales, Brian A. White, ...","Electromagnetic Productions, Sculptor Media, n...",An aging ex-boxer gangster (Liam Neeson) w...
3,Armor,HD,"Jeff Chase, Joel Cohen, Josh Whites, Josh Wigg...","Convergence Entertainment, TPC, BondIt Media C...",Armored truck security guard James Brody i...
4,Elevation,HD,"Anthony Mackie, Drexel Malkoff, Shauna Earp, M...","Lyrical Media, Fuller Media, Inspire Entertain...",A single father and two women venture from...
...,...,...,...,...,...
91,NEXT at the Kennedy Center,,,,"""NEXT at the Kennedy Center,"" a new primet..."
92,Entergalactic,,"Laura Harrier, Jessica Williams, Timothée Chal...",Khalabo Ink Society,Ambitious artist Jabari attempts to balanc...
93,Mirror Mirror,,"Santi Millán, Carlos Areces, Natalia de Molina...",Netflix,"In this comedic coming-of-age story, an ad..."
94,Murders at the Burger Joint,,,,"Speedway, Indiana, is famous for its fast ..."
